# 1 Imports

In [2]:
import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
from tqdm import tqdm


In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# 2 Data loading

In [4]:
links = ['http://tennis-data.co.uk/2000/2000.xls', 'http://tennis-data.co.uk/2001/2001.xls', 'http://tennis-data.co.uk/2002/2002.xls', 'http://tennis-data.co.uk/2003/2003.xls', 
         'http://tennis-data.co.uk/2004/2004.xls', 'http://tennis-data.co.uk/2005/2005.xls', 'http://tennis-data.co.uk/2006/2006.xls', 'http://tennis-data.co.uk/2007/2007.xls', 
         'http://tennis-data.co.uk/2008/2008.zip', 'http://tennis-data.co.uk/2009/2009.xls', 'http://tennis-data.co.uk/2010/2010.xls', 'http://tennis-data.co.uk/2011/2011.xls', 
         'http://tennis-data.co.uk/2012/2012.xls', 'http://tennis-data.co.uk/2013/2013.xlsx', 'http://tennis-data.co.uk/2014/2014.xlsx', 'http://tennis-data.co.uk/2015/2015.xlsx', 
         'http://tennis-data.co.uk/2016/2016.xlsx', 'http://tennis-data.co.uk/2017/2017.xlsx', 'http://tennis-data.co.uk/2018/2018.xlsx', 'http://tennis-data.co.uk/2019/2019.xlsx', 
         'http://tennis-data.co.uk/2020/2020.xlsx', 'http://tennis-data.co.uk/2021/2021.xlsx', 'http://tennis-data.co.uk/2022/2022.xlsx', 'http://tennis-data.co.uk/2023/2023.xlsx',
         'http://tennis-data.co.uk/2024/20234.xlsx']

## 2.1 Merge files into one dataframe

In [7]:
df = pd.DataFrame()
for i, elem in enumerate(links):
    if elem[-4:] == '.zip':
        content = requests.get(elem)
        zf = zipfile.ZipFile(BytesIO(content.content))
        temp = pd.read_excel(zf.open(zf.namelist()[0])) 
    else:
        temp = pd.read_excel(elem) 
    df = pd.concat([df, temp], ignore_index=True)

/Users/joseguzman/Library/CloudStorage/OneDrive-nwmissouri.edu/Summer1/Summer2/Capstone_Project/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/joseguzman/Library/CloudStorage/OneDrive-nwmissouri.edu/Summer1/Summer2/Capstone_Project/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/joseguzman/Library/CloudStorage/OneDrive-nwmissouri.edu/Summer1/Summer2/Capstone_Project/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/joseguzman/Library/CloudStorage/OneDrive-nwmissouri.edu/Summer1/Summer2/Capstone_Project/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# 3 Data preprocessing

In [8]:
df['Best of'] = df['Best of'].fillna(3)
df = df[df['Comment'] == 'Completed'].reset_index(drop=True)
df = df[~df['WRank'].isnull()].reset_index(drop=True)
df = df[~df['LRank'].isnull()].reset_index(drop=True)
df = df[~df['W1'].isnull()].reset_index(drop=True)
df = df[~df['W2'].isnull()].reset_index(drop=True)
df = df[~df['L1'].isnull()].reset_index(drop=True)
df = df[~df['L2'].isnull()].reset_index(drop=True)
df[['W3', 'W4', 'W5', 'L3', 'L4', 'L5']] = df[['W3', 'W4', 'W5', 'L3', 'L4', 'L5']].fillna(0)

In [9]:
df['B365W'] = df['B365W'].fillna(df[['CBW', 'GBW', 'IWW', 'SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW']].mean(axis = 1)).fillna(df['AvgW'])
df['B365L'] = df['B365L'].fillna(df[['CBL', 'GBL', 'IWL', 'SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL']].mean(axis = 1)).fillna(df['AvgL'])

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [10]:
df['ind'] = [(lambda x: x % 2)(x) for x in range(len(df))]

In [11]:
def checkempty(str):
    if str == ' ':
        return 0
    return str

In [12]:
df['W3'] = df['W3'].apply(checkempty)
df['L3'] = df['L3'].apply(checkempty)
df[['W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5']] = df[['W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5']].astype(float).astype(int)

In [13]:
df['Player_1'] = df.apply(lambda row: row['Winner'] if row['ind'] == 0 else row['Loser'], axis = 1)
df['Player_2'] = df.apply(lambda row: row['Winner'] if row['ind'] == 1 else row['Loser'], axis = 1)
df['Rank_1'] = df.apply(lambda row: row['WRank'] if row['ind'] == 0 else row['LRank'], axis = 1)
df['Rank_2'] = df.apply(lambda row: row['WRank'] if row['ind'] == 1 else row['LRank'], axis = 1)
df['Pts_1'] = df.apply(lambda row: row['WPts'] if row['ind'] == 0 else row['LPts'], axis = 1)
df['Pts_2'] = df.apply(lambda row: row['WPts'] if row['ind'] == 1 else row['LPts'], axis = 1)
df['Odd_1'] = df.apply(lambda row: row['B365W'] if row['ind'] == 0 else row['B365L'], axis = 1)
df['Odd_2'] = df.apply(lambda row: row['B365W'] if row['ind'] == 1 else row['B365L'], axis = 1)

In [14]:
def score(df):
    if df['ind'] == 0:
        return str(df['W1']) + '-' + str(df['L1']) + ' ' + str(df['W2']) + '-' + str(df['L2']) + ' ' + str(df['W3']) + '-' + str(df['L3']) +\
 ' ' + str(df['W4']) + '-' + str(df['L4']) + ' ' + str(df['W5']) + '-' + str(df['L5']) + ' '
    return str(df['L1']) + '-' + str(df['W1']) + ' ' + str(df['L2']) + '-' + str(df['W2']) + ' ' + str(df['L3']) + '-' + str(df['W3']) +\
 ' ' + str(df['L4']) + '-' + str(df['W4']) + ' ' + str(df['L5']) + '-' + str(df['W5']) + ' '

In [15]:
df['Score'] = df.apply(lambda row: score(row).replace('0-0', ''), axis = 1)

In [16]:
df['Score'] = df['Score'].apply(lambda x: x.strip())

In [17]:
new_df = df[['Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Player_1', 'Player_2','Winner', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2', 'Score']]

In [18]:
def check(str):
    return str.replace('NR', -1)

In [19]:
new_df['Date'] = pd.to_datetime(new_df['Date'], format = '%Y-%M-%d')
new_df = new_df.fillna(-1)
new_df[['Best of', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2']] = new_df[['Best of', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2']].apply(check)
new_df[['Best of', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2']] = new_df[['Best of', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2']].astype(float).astype(int)

/var/folders/jn/mg3jxbw96_v18x44tq5dfhv00000gn/T/ipykernel_4410/115323928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date'] = pd.to_datetime(new_df['Date'], format = '%Y-%M-%d')
/var/folders/jn/mg3jxbw96_v18x44tq5dfhv00000gn/T/ipykernel_4410/50769503.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return str.replace('NR', -1)


In [20]:
new_df.to_csv('atp_tennis.csv', index=False)